# Streams

## Stream source

In [1]:
import java.util.stream.Stream;
import java.util.Arrays;

Stream<String> empty = Stream.empty();
Stream<String> single = Stream.of("one");
Stream<String> fromArray = Stream.of("one", "two", "three");
Stream<Integer> fromList = Arrays.asList(1, 2, 3).stream();
Stream<Double> randoms = Stream.generate(Math::random); // infinite
Stream<Integer> oddNumbers = Stream.iterate(1, n -> n + 2); // infinite

null

## Intermediate operations

* Stream filtering
```java
Stream<T> filter(Predicate<? super T> predicate)
```

In [23]:
import java.util.stream.Stream;

Stream<String> stream = Stream.of("jeans", "dress", "coat", "suit");
stream.filter(s -> s.startsWith("d")).forEach(System.out::println);

dress


null

* Removing duplicates
```java
Stream<T> distinct()
```

In [24]:
import java.util.stream.Stream;

Stream<String> stream = Stream.of("jeans", "dress", "dress", "jeans");
stream.distinct().forEach(System.out::println);

jeans
dress


null

* Limit and skip
```java
Stream<T> limit(int maxSize)
Stream<T> skip(int n)
```

In [26]:
import java.util.stream.Stream;

Stream<Integer> stream = Stream.iterate(1, n -> n + 2);
stream.skip(4).limit(2).forEach(System.out::println);

9
11


null

* Data transformation by applying function to each element of stream
```java
<R> Stream<R> map(Function<? super T, ? extends R> mapper)
```

In [28]:
import java.util.stream.Stream;

Stream<String> stream = Stream.of("jeans", "dress", "coat", "suit");
stream.map(String::length).forEach(System.out::println);

5
5
4
4


null

* Data transformation producing stream from each element of the stream and then combining them
```java
<R> Stream<R> flatMap(Function<? super T, ? extends Stream<? extends R>> mapper)
```

In [30]:
import java.util.stream.Stream;
import java.util.*;

List<String> zero = Arrays.asList();
List<String> one = Arrays.asList("suit");
List<String> two = Arrays.asList("dress", "jeans");
Stream<List<String>> wearings = Stream.of(zero, one, two);

wearings.flatMap(List::stream).forEach(System.out::println);

suit
dress
jeans


null

* Sorting stream 
Requires all of the data, so it can't be used with infinite streams.
```java
Stream<T> sorted()
Stream<T> sorted(Comparator<? super T> comparator)
```

In [42]:
import java.util.stream.Stream;
import java.util.Comparator;

System.out.println("Sorted:");
Stream<String> stream = Stream.of("jeans", "dress", "coat", "suit");
stream.sorted().forEach(System.out::println);

System.out.println("Reversed:");
stream = Stream.of("jeans", "dress", "coat", "suit");
stream.sorted(Comparator.reverseOrder()).forEach(System.out::println);

Sorted:
coat
dress
jeans
suit
Reversed:
suit
jeans
dress
coat


null

* Perform operation whithout changing a stream (usefull for debugging)
```java
Stream<T> peek(Consumer<? super T> action)
```

In [51]:
import java.util.stream.Stream;
import java.util.Comparator;

Stream<String> stream = Stream.of("jeans", "dress", "coat", "suit");
long count = stream.filter(s -> s.startsWith("d")).
    peek(System.out::println).count();

return count;


dress


1

## Terminal operations
No operations performed with stream until terminal operation is called.

* Count elements of stream.
```java
long count()
```

In [2]:
import java.util.stream.Stream;

Stream<Integer> ints = Stream.of(1,3,4,5,6,7);
return ints.count();

6

* Find min and max in stream.
```java
Optional<T> min(<? super T> comparator)
Optional<T> max(<? super T> comparator)
```

In [3]:
import java.util.stream.Stream;
import java.util.Optional;

Stream<String> s = Stream.of("jeans", "dress", "coat");
Optional<String> min = s.min((s1, s2) -> s1.length() - s2.length());
return min;

Optional[coat]

* Getting single element of stream.
```java
Optional<T> findAny()
Optional<T> findFirst()
```

In [4]:
import java.util.stream.Stream;

return Stream.generate(() -> "tick").findAny();

Optional[tick]

In [5]:
import java.util.stream.Stream;

return Stream.of(23, 43, 56).findFirst();

Optional[23]

* Check elements of stream with predicate
```java
boolean anyMatch(Predicate<? super T> predicate)
boolean allMatch(Predicate<? super T> predicate)
boolean noneMatch(Predicate<? super T> predicate)
```

In [6]:
import java.util.stream.Stream;
import java.util.function.Predicate;

Stream<String> infinite = Stream.generate(() -> "chimp");
Predicate<String> p = s -> Character.isLetter(s.charAt(0));
return infinite.noneMatch(p);

false

* Looping through stream elements
```java
void forEach(Consumer<? super T> action)
```

In [7]:
import java.util.stream.Stream;

Stream.of("red", "orange", "yellow", "green", "blue", "violet").
        forEach(System.out::println);

red
orange
yellow
green
blue
violet


null

* Combine elements to single object 
```java
T reduce(T identity, BinaryOperation<T> accumulator)
Optional<T> reduce(BinaryOperation<T> accumulator)
<U> U reduce(U identity, BiFunction<U, ? super T, U> accumulator, BinaryOperator<U> combiner) // for parallel streams
```

In [8]:
import java.util.stream.Stream;

return Stream.of("w", "o", "l", "f").reduce("", (s, c) -> s + c);


wolf

In [9]:
import java.util.stream.Stream;

return Stream.of(2, 4, 6).reduce((a, b) -> a * b);

Optional[48]

In [10]:
import java.util.stream.Stream;
import java.util.function.BinaryOperator;

BinaryOperator<Integer> op = (a, b) -> a * b;
return Stream.of(2, 4, 6).reduce(1, op, op);

48

* *Mutable reduction* with collect. Uses the same mutable object while accumulating.
```java
<R> R collect(Supplier<R> supplier, BiConsumer<R, ? super T> accumulator, BiConsumer<R, R> combiner)
<R, A> R collect(Collector<? super T, A, R> collector)
```

In [13]:
import java.util.stream.Stream;

Stream<String> stream = Stream.of("W", "o", "l", "f");
StringBuilder word = stream.collect(StringBuilder::new, 
                                    StringBuilder::append, 
                                    StringBuilder::append);
return word.toString();

Wolf

Example below shows `collect` for parallel streams. 3rd argument is a combiner which adds all of the elements of one TreeSet to another merging results of parallel operations.

In [17]:
import java.util.stream.Stream;
import java.util.TreeSet;

Stream<String> stream = Stream.of("w", "o", "l", "f");
TreeSet<String> set = stream.collect(TreeSet::new, 
                                    TreeSet::add, 
                                    TreeSet::addAll);
return set.toString();

[f, l, o, w]

Examples below shows the usage of builtin Java collectors.

In [20]:
import java.util.stream.*;
import java.util.TreeSet;

Stream<String> stream = Stream.of("w", "o", "l", "f");
TreeSet<String> set = stream.collect(
    Collectors.toCollection(TreeSet::new));
return set.toString();

[f, l, o, w]

In [22]:
import java.util.stream.*;
import java.util.Set;

Stream<String> stream = Stream.of("w", "o", "l", "f");
Set<String> set = stream.collect(Collectors.toSet());
return set.toString();

[f, w, l, o]

## Primitive Stream sources

Three types of primitive streams:
`IntStream` - `int`, `short`, `byte`, `char`
`LongStream` - `long`
`DoubleStream` - `double`, `float`
These classes also have `empty`, `of`, `generate` and `iterate` methods just like class `Stream`.
`IntStream` also has `range` and `rangeClosed` methods.

In [1]:
import java.util.stream.IntStream;

IntStream.range(1, 6).forEach(System.out::println);

1
2
3
4
5


null

In [2]:
import java.util.stream.IntStream;

IntStream.rangeClosed(1, 5).forEach(System.out::println);

1
2
3
4
5


null

Class `Stream` has methods to convert to each of the primitive streams - `mapToInt`, `mapToLong` and `mapToDouble`.

Each of the classes for primitive streams have `mapToObj` method to convert to `Stream` and `mapToInt`, `mapToLong` and `mapToDouble` to convert to other primitive stream type. Except that `IntStream` (`LongStream`, `DoubleStream`) has `map` method instead of `mapToInt` (`mapToLong`, `mapToDouble`). 

For each of conversions corresponding function is used (for `<Type>` in (`Int`, `Long`, `Double`)):
`To<Type>Function` - to convert from `Stream` to `<Type>Stream`,  
`<Type>Function` - to convert from `<Type>Stream` to `Stream`,  
`<SourceType>To<TargetType>Function` - to convert from `<SourceType>Stream` to `<TargetType>Stream`,  
and `<Type>UnaryOperator` to perform operations without changing type of stream.

In [5]:
import java.util.stream.*;
import java.util.function.*;

Stream<String> colors = Stream.of("red", "orange", "yellow", "green", "blue", "violet");
ToIntFunction<String> mapper = String::length;
colors.mapToInt(mapper).forEach(System.out::println);

3
6
6
5
4
6


null

## Optional with Primitive Streams

Class `Optional` also has its own versions for primitives:  
`OptionalInt` - with method `getAsInt`,  
`OptionalLong` - with method `getAsLong`,  
`OptionalDouble` - with method `getAsDouble`.

In [8]:
import java.util.stream.*;

LongStream.of(4, 7, 10).min().ifPresent(System.out::println);
LongStream.of(4, 7, 10).average().ifPresent(System.out::println);

4
7.0


null

## Summary Statistics

In [13]:
import java.util.stream.*;
import java.util.*;

IntSummaryStatistics stats = IntStream.of(1, 4, 6, 11).summaryStatistics();
return stats.getMax() - stats.getMin();

10

## Functional Inerfaces for Primitives

### Supplier
* `BooleanSupplier` - `boolean getAsBoolean()`
* `IntSupplier` - `int getAsInt()`
* `LongSupplier` - `long getAsLong()`
* `DoubleSupplier` - `double getAsDouble()`

### Consumer
* `IntConsumer` - `void accept(int i)`
* `LongConsumer` - `void accept(long l)`
* `DoubleConsumer` - `void accept(double d)`  

### BiConsumer analogs
* `ObjIntConsumer<T>` - `void accept(T t, int i)`
* `ObjLongConsumer<T>` - `void accept(T t, long l)`
* `ObjDoubleConsumer<T>` - `void accept(T t, double d)`

### Predicate
* `IntPredicate` - `boolean test(int i)`
* `LongPredicate` - `boolean test(long l)`
* `DoublePredicate` - `boolean test(double d)`

### Function
* `IntFunction<R>` - `R apply(int i)`
* `LongFunction<R>` - `R apply(long l)`
* `DoubleFunction<R>` - `R apply(double d)`
* `ToIntFunction<T>` - `int applyAsInt(T t)`
* `ToLongFunction<T>` - `long applyAsLong(T t)`
* `ToDoubleFunction<T>` - `double applyAsDouble(T t)`
* `IntToLongFunction` - `long applyAsLong(int i)`
* `IntToDoubleFunction` - `double applyAsDouble(int i)`
* `LongToIntFunction` - `int applyAsInt(long l)`
* `LongToDoubleFunction` - `double applyAsDouble(long l)`
* `DoubleToIntFunction` - `int applyAsInt(double d)`
* `DoubleToLongFunction` - `long applyAsLong(double d)`

### BiFunction analogs
* `ToIntBiFunction<T, U>` - `int applyAsInt(T t, U u)`
* `ToLongBiFunction<T, U>` - `long applyAsLong(T t, U u)`
* `ToDoubleBiFunction<T, U>` - `double applyAsDouble(T t, U u)`

### UnaryOperator
* `IntUnaryOperator` - `int applyAsInt(int i)`
* `LongUnaryOperator` - `long applyAsLong(long l)`
* `DoubleUnaryOperator` - `double applyAsDouble(double d)`

### BinaryOperator
* `IntBinaryOperator` - `int applyAsInt(int i1, int i2)`
* `LongBinaryOperator` - `long applyAsLong(long l1, long l2)`
* `DoubleBinaryOperator` - `double applyAsDouble(double d1, double d2)`